# Importing and retrieving data

In [27]:
import DataRetriever as dr
import pandas as pd
import plotly.express as px
import numpy as np

retriever = dr.DataRetriever()

# Load the metadata
metadata = retriever.get_data("metadata-year2.pkl")
metadata.rename(columns={"Unnamed: 0": "Attribute"}, inplace=True)

#vLoad the NZERTF data
year_2 = retriever.get_data("All-Subsystems-minute-Year2.pkl")

# Add timestamp converted to UTC as index to year_2
# year_2["Timestamp UTC"] = pd.to_datetime(year_2["Timestamp"], utc=True, infer_datetime_format=True)
# year_2.set_index("Timestamp UTC", inplace=True)

columns_to_clip = year_2.select_dtypes(np.number).columns #Only integer columns not string.
year_2[columns_to_clip].clip(lower=0) #Minimum value of column must be 0, else set to 0.

,TimeStamp_Count,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,...,SHW_GlycolFlowHXCoriolisSHW,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow
0,1,0.000000,4.994715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,184.412,1108.765,4.471,72.102,74.655,0.0
1,2,0.000000,5.046792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,184.618,1097.577,4.508,72.096,74.701,0.0
2,3,0.000000,5.129493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,169.844,1086.113,4.510,72.077,74.759,0.0
3,4,0.001981,5.163539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,167.300,1079.586,4.529,72.074,74.829,0.0
4,5,0.001981,5.232744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,182.976,1048.502,4.438,72.086,74.863,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519599,519600,1.757590,4.512704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,419.930636,942.630328,0.001164,0.000296,11.725,22.563,4.265,67.645,68.550,0.0
519600,519601,1.760893,4.514255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,419.930636,942.630328,0.001546,0.000268,11.178,22.475,4.286,67.655,68.551,0.0
519601,519602,1.763535,4.548853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,419.930636,942.630328,0.000187,0.000000,10.767,22.388,4.293,67.644,68.549,0.0
519602,519603,1.763535,4.557287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,419.930636,942.630328,0.001145,0.000000,10.903,22.169,4.306,67.651,68.552,0.0


In [28]:
watt_attributes = metadata[metadata["Units"] == "W"] # We only want attributes measured in Watts
watt_attributes = watt_attributes[watt_attributes['max_value'] > 0] #If no measurements are recorded in the attribute, we dont need to include it
watt_attributes = watt_attributes[watt_attributes["Parameter"] != "Power_Thermal"]
watt_attributes = watt_attributes[watt_attributes["Subsystem"] != "PV"]

consuming_attributes = watt_attributes["Attribute"].to_list()
unwanted_attributes = ["Elec_PowerClothesWasher", "Elec_PowerRefrigerator",  "Elec_PowerMicrowave", "Elec_PowerPlugsLR", "Elec_PowerLights2ndFloor", "Elec_PowerPlugsBR3", "Elec_PowerPlugsBR2", "Elec_PowerPlugsInstKitD"]
consuming_attributes = [attribute for attribute in consuming_attributes if attribute not in unwanted_attributes]

['Load_LRPlugLoadsPowerUsage', 'Load_KPlugLoadsPowerUsage', 'Load_MBRPlugLoadsPowerUsage', 'Load_BR2PlugLoadsPowerUsage', 'Load_BR3PlugLoadsPowerUsage', 'Load_BR4PlugLoadsPowerUsage', 'Load_1stFloorLightsPowerUsage', 'Load_2ndFloorLightsPowerUsage', 'Load_DryerPowerTotal', 'Load_OvenPowerTotal', 'Load_BasementPlugLoadsPowerUsage', 'Load_ClothesWasherPowerWithStandby', 'Load_RefrigeratorPowerWithStandby', 'Load_MicrowavePowerWithStandby', 'Elec_PowerPlugsBaseAHeliodyneHXs', 'Elec_PowerPlugsBaseB', 'Elec_PowerPlugsBaseC', 'Elec_PowerSumpPump', 'Elec_PowerLightsBasementStair', 'Elec_PowerSpare1', 'Elec_PowerPlugsDR', 'Elec_PowerPlugsBR4', 'Elec_PowerPlugsEntryHall', 'Elec_PowerDryer1of2', 'Elec_PowerDryer2of2', 'Elec_PowerHRV', 'Elec_PowerHeatPumpWH2of2', 'Elec_PowerMakeUpAirDamper', 'Elec_PowerPlugsMBAEast', 'Elec_PowerPlugsMBAWest', 'Elec_PowerPlugsKitSink', 'Elec_PowerGarbageDisposal', 'Elec_PowerPlugsKitRange', 'Elec_PowerPlugsKitPeninsula', 'Elec_PowerPlugs2ndFloor', 'Elec_PowerSpare

In [29]:
energy_data = year_2[["Timestamp"] + consuming_attributes]

In [30]:
total_consumed = energy_data[consuming_attributes].sum().sum()
print(total_consumed * (1/60) / 1000)

16431.889886245914
